In [3]:
import tensorflow as tf
import numpy as np
from zipfile import ZipFile
import os


In [4]:
# Define hyperparameters
AUTOTUNE = tf.data.AUTOTUNE
BUFFER_SIZE = 1000
BATCH_SIZE = 1
IMG_WIDTH = 256
IMG_HEIGHT = 256
LAMBDA = 10
EPOCHS = 25

In [5]:
def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = (tf.cast(image, tf.float32) / 127.5) - 1
    image = tf.reshape(image, [IMG_HEIGHT, IMG_WIDTH, 3])
    return image

def read_tfrecord(example):
    tfrecord_format = {
        'image': tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example['image'])
    return image

def load_dataset(filenames):
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.map(read_tfrecord, num_parallel_calls=AUTOTUNE)
    return dataset

In [8]:
def downsample(filters, size, apply_norm=True):
    initializer = tf.random_normal_initializer(0., 0.02)
    result = tf.keras.Sequential()
    
    result.add(tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                                     kernel_initializer=initializer, use_bias=False))
    
    if apply_norm:
        result.add(tf.keras.layers.LayerNormalization())
    
    result.add(tf.keras.layers.LeakyReLU())
    return result

def upsample(filters, size, apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)
    result = tf.keras.Sequential()
    
    result.add(tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                              padding='same',
                                              kernel_initializer=initializer,
                                              use_bias=False))
    
    result.add(tf.keras.layers.LayerNormalization())
    
    if apply_dropout:
        result.add(tf.keras.layers.Dropout(0.5))
    
    result.add(tf.keras.layers.ReLU())
    return result

In [9]:
def Generator():
    inputs = tf.keras.layers.Input(shape=[IMG_HEIGHT, IMG_WIDTH, 3])
    
    # Encoder
    down_stack = [
        downsample(64, 4, apply_norm=False),
        downsample(128, 4),
        downsample(256, 4),
        downsample(512, 4),
        downsample(512, 4),
        downsample(512, 4),
        downsample(512, 4),
        downsample(512, 4),
    ]
    
    # Decoder
    up_stack = [
        upsample(512, 4, apply_dropout=True),
        upsample(512, 4, apply_dropout=True),
        upsample(512, 4, apply_dropout=True),
        upsample(512, 4),
        upsample(256, 4),
        upsample(128, 4),
        upsample(64, 4),
    ]
    
    initializer = tf.random_normal_initializer(0., 0.02)
    last = tf.keras.layers.Conv2DTranspose(3, 4,
                                         strides=2,
                                         padding='same',
                                         kernel_initializer=initializer,
                                         activation='tanh')
    
    x = inputs
    
    # Downsampling through the model
    skips = []
    for down in down_stack:
        x = down(x)
        skips.append(x)
    
    skips = reversed(skips[:-1])
    
    # Upsampling and establishing the skip connections
    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = tf.keras.layers.Concatenate()([x, skip])
    
    x = last(x)
    
    return tf.keras.Model(inputs=inputs, outputs=x)


In [17]:
def Discriminator():
    initializer = tf.random_normal_initializer(0., 0.02)
    
    inp = tf.keras.layers.Input(shape=[IMG_HEIGHT, IMG_WIDTH, 3], name='input_image')
    
    x = inp
    
    down1 = downsample(64, 4, False)(x)
    down2 = downsample(128, 4)(down1)
    down3 = downsample(256, 4)(down2)
    
    zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3)
    conv = tf.keras.layers.Conv2D(512, 4, strides=1,
                                kernel_initializer=initializer,
                                use_bias=False)(zero_pad1)
    
    norm1 = tf.keras.layers.LayerNormalization()(conv)
    leaky_relu = tf.keras.layers.LeakyReLU()(norm1)
    
    zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu)
    
    last = tf.keras.layers.Conv2D(1, 4, strides=1,
                                kernel_initializer=initializer)(zero_pad2)
    
    return tf.keras.Model(inputs=inp, outputs=last)

In [19]:
@tf.function
def train_step(real_x, generator_g, generator_f, discriminator_x, discriminator_y,
               generator_g_optimizer, generator_f_optimizer,
               discriminator_x_optimizer, discriminator_y_optimizer, generator_loss, discriminator_loss, calc_cycle_loss, identity_loss):
    with tf.GradientTape(persistent=True) as tape:
        fake_y = generator_g(real_x, training=True)
        cycled_x = generator_f(fake_y, training=True)
        
        fake_x = generator_f(real_x, training=True)
        cycled_y = generator_g(fake_x, training=True)
        
        # Identity loss
        same_x = generator_f(real_x, training=True)
        same_y = generator_g(real_x, training=True)
        
        disc_real_x = discriminator_x(real_x, training=True)
        disc_fake_x = discriminator_x(fake_x, training=True)
        
        disc_real_y = discriminator_y(real_x, training=True)
        disc_fake_y = discriminator_y(fake_y, training=True)
        
        # Calculate losses
        gen_g_loss = generator_loss(disc_fake_y, fake_y, real_x)
        gen_f_loss = generator_loss(disc_fake_x, fake_x, real_x)
        
        total_cycle_loss = calc_cycle_loss(real_x, cycled_x) + calc_cycle_loss(real_x, cycled_y)
        
        total_gen_g_loss = gen_g_loss + total_cycle_loss + identity_loss(real_x, same_y)
        total_gen_f_loss = gen_f_loss + total_cycle_loss + identity_loss(real_x, same_x)
        
        disc_x_loss = discriminator_loss(disc_real_x, disc_fake_x)
        disc_y_loss = discriminator_loss(disc_real_y, disc_fake_y)
    
    # Calculate gradients
    generator_g_gradients = tape.gradient(total_gen_g_loss, generator_g.trainable_variables)
    generator_f_gradients = tape.gradient(total_gen_f_loss, generator_f.trainable_variables)
    
    discriminator_x_gradients = tape.gradient(disc_x_loss, discriminator_x.trainable_variables)
    discriminator_y_gradients = tape.gradient(disc_y_loss, discriminator_y.trainable_variables)
    
    # Apply gradients
    generator_g_optimizer.apply_gradients(zip(generator_g_gradients, generator_g.trainable_variables))
    generator_f_optimizer.apply_gradients(zip(generator_f_gradients, generator_f.trainable_variables))
    
    discriminator_x_optimizer.apply_gradients(zip(discriminator_x_gradients,
                                                discriminator_x.trainable_variables))
    discriminator_y_optimizer.apply_gradients(zip(discriminator_y_gradients,
                                                discriminator_y.trainable_variables))
    
    return total_gen_g_loss, total_gen_f_loss, disc_x_loss, disc_y_loss

In [13]:
def generate_images(model, test_input):
    prediction = model(test_input)
    return prediction

In [21]:
def generator_loss(disc_generated_output, gen_output, target):
    gan_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    total_loss = gan_loss(tf.ones_like(disc_generated_output), disc_generated_output)
    
    l1_loss = tf.reduce_mean(tf.abs(target - gen_output))
    total_loss += LAMBDA * l1_loss
    return total_loss

def discriminator_loss(disc_real_output, disc_generated_output):
    real_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(disc_real_output), disc_real_output)
    generated_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.zeros_like(disc_generated_output), disc_generated_output)
    total_loss = real_loss + generated_loss
    return total_loss

def calc_cycle_loss(real_image, cycled_image):
    loss1 = tf.reduce_mean(tf.abs(real_image - cycled_image))
    return LAMBDA * loss1

def identity_loss(real_image, same_image):
    loss = tf.reduce_mean(tf.abs(real_image - same_image))
    return LAMBDA * 0.5 * loss

def main():
    # Load datasets
    monet_files = tf.io.gfile.glob('./gan-getting-started/monet_tfrec/*.tfrec')
    photo_files = tf.io.gfile.glob('./gan-getting-started/photo_tfrec/*.tfrec')
    
    monet_data = load_dataset(monet_files)
    photo_data = load_dataset(photo_files)
    
    # Take first, then cache and shuffle
    monet_data = monet_data.take(300).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    photo_data = photo_data.take(300).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    
    # Create models and optimizers
    generator_g = Generator()
    generator_f = Generator()
    
    discriminator_x = Discriminator()
    discriminator_y = Discriminator()
    
    generator_g_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
    generator_f_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
    
    discriminator_x_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
    discriminator_y_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
    
    # Training loop
    for epoch in range(EPOCHS):
        # Initialize loss variables for each epoch
        gen_g_loss = gen_f_loss = disc_x_loss = disc_y_loss = 0
        
        # Track number of batches
        n_batches = 0
        
        for photo, monet in tf.data.Dataset.zip((photo_data, monet_data)):
            batch_gen_g_loss, batch_gen_f_loss, batch_disc_x_loss, batch_disc_y_loss = train_step(
                photo, generator_g, generator_f,
                discriminator_x, discriminator_y,
                generator_g_optimizer, generator_f_optimizer,
                discriminator_x_optimizer, discriminator_y_optimizer, 
                generator_loss, discriminator_loss, calc_cycle_loss, identity_loss)
            
            # Accumulate losses
            gen_g_loss += batch_gen_g_loss
            gen_f_loss += batch_gen_f_loss
            disc_x_loss += batch_disc_x_loss
            disc_y_loss += batch_disc_y_loss
            n_batches += 1
        
        # Calculate average losses for the epoch
        if n_batches > 0:
            gen_g_loss /= n_batches
            gen_f_loss /= n_batches
            disc_x_loss /= n_batches
            disc_y_loss /= n_batches
            
            print(f'Epoch {epoch + 1}: Gen G Loss = {gen_g_loss:.4f}, Gen F Loss = {gen_f_loss:.4f}, '
                  f'Disc X Loss = {disc_x_loss:.4f}, Disc Y Loss = {disc_y_loss:.4f}')

    # Generate images for submission
    os.makedirs('generated_images', exist_ok=True)
    
    for i, photo in enumerate(photo_data.take(7500)):
        generated_image = generate_images(generator_g, photo)
        generated_image = ((generated_image[0] + 1) * 127.5).numpy().astype(np.uint8)
        tf.keras.preprocessing.image.save_img(
            f'generated_images/generated_{i}.jpg',
            generated_image
        )
    
    # Create submission zip file
    with ZipFile('images.zip', 'w') as zipf:
        for root, dirs, files in os.walk('generated_images'):
            for file in files:
                zipf.write(os.path.join(root, file))

In [22]:

if __name__ == "__main__":
    main()

2025-01-21 22:33:25.076694: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:370] TFRecordDataset `buffer_size` is unspecified, default to 262144
2025-01-21 22:39:37.239324: W tensorflow/core/kernels/data/cache_dataset_ops.cc:914] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


Epoch 1: Gen G Loss = 7.9639, Gen F Loss = 7.9866, Disc X Loss = 1.4831, Disc Y Loss = 1.4701


2025-01-21 22:45:08.818458: W tensorflow/core/kernels/data/cache_dataset_ops.cc:914] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-01-21 22:45:09.768130: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 2: Gen G Loss = 5.1651, Gen F Loss = 5.1572, Disc X Loss = 1.3644, Disc Y Loss = 1.3525


2025-01-21 22:50:52.803194: W tensorflow/core/kernels/data/cache_dataset_ops.cc:914] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


...

Epoch 23: Gen G Loss = 2.6523, Gen F Loss = 2.6427, Disc X Loss = 1.3850, Disc Y Loss = 1.3853


2025-01-22 01:37:37.263206: W tensorflow/core/kernels/data/cache_dataset_ops.cc:914] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


Epoch 24: Gen G Loss = 2.6029, Gen F Loss = 2.6042, Disc X Loss = 1.3808, Disc Y Loss = 1.3854


2025-01-22 01:44:58.873725: W tensorflow/core/kernels/data/cache_dataset_ops.cc:914] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


Epoch 25: Gen G Loss = 2.6615, Gen F Loss = 2.6536, Disc X Loss = 1.3965, Disc Y Loss = 1.3831
